In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mywaste/WasteImagesDataset/Glass/glass (124).jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass141.jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass (236).jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass (50).jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass374.jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass (196).jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass184.jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass75.jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass409.jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass207.jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass433.jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass311.jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass (94).jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass (15).jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass185.jpg
/kaggle/input/mywaste/WasteImagesDataset/Glass/glass342.jpg
/kaggle/input/mywaste/Wast

In [18]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [20]:
input_shape = (224, 224, 3)

In [22]:
base_model = MobileNet(include_top=False, input_shape=input_shape, weights = "imagenet")

In [23]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(9, activation='softmax')(x)

In [24]:
model = Model(inputs=base_model.input, outputs=x)

In [26]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [27]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gen_train = ImageDataGenerator(rescale = 1/255, shear_range = 0.2, zoom_range = 0.2, 
                               brightness_range = (0.1, 0.5), horizontal_flip=True)

train_data = gen_train.flow_from_directory("/kaggle/input/mywaste/WasteImagesDataset",
                                           target_size = (224, 224), batch_size = 32, class_mode="categorical")

Found 5957 images belonging to 9 classes.


In [29]:
result = model.fit_generator(train_data, epochs = 30, steps_per_epoch=20)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/30
18/20 [==========================>...] - ETA: 1s - loss: 1.5964 - accuracy: 0.5434

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:980: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


20/20 [==============================] - 25s 639ms/step - loss: 1.5590 - accuracy: 0.5453
Epoch 2/30
20/20 [==============================] - 11s 561ms/step - loss: 1.1578 - accuracy: 0.6375
Epoch 3/30
20/20 [==============================] - 12s 593ms/step - loss: 1.0350 - accuracy: 0.6797
Epoch 4/30
20/20 [==============================] - 11s 535ms/step - loss: 0.9618 - accuracy: 0.7125
Epoch 5/30
20/20 [==============================] - 12s 581ms/step - loss: 0.8005 - accuracy: 0.7297
Epoch 6/30
20/20 [==============================] - 12s 572ms/step - loss: 0.7336 - accuracy: 0.7688
Epoch 7/30
20/20 [==============================] - 11s 547ms/step - loss: 0.6476 - accuracy: 0.7719
Epoch 8/30
20/20 [==============================] - 11s 562ms/step - loss: 0.6761 - accuracy: 0.7656
Epoch 9/30
20/20 [==============================] - 11s 565ms/step - loss: 0.6039 - accuracy: 0.8141
Epoch 10/30
20/20 [==============================] - 13s 631ms/step - loss: 0.5720 - accuracy: 0.8172


In [30]:
model.save("epoch30.h5")

In [14]:
model =tf.keras.models.load_model("/kaggle/working/epoch30.h5",compile=False)

#Convert to tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [32]:
converter.optimizations = [tf.lite.Optimize.DEFAULT] #Uses default optimization strategy to reduce the model size
#converter.target_spec.supported_types = [tf.uint8]
tflite_model = converter.convert()
open("epocho30.tflite", "wb").write(tflite_model)

3389376